# Neural Machine Translation for Cantonese-English Language Pair
Florence Yuen
- Uses datasets from Tatoeba and OpenSubtitles to load Cantonese-English language pair data
- Preprocess character-based and Jyutping romanized Cantonese data by tokenizing and doing data cleansing
- Apply mBART-50 pre-trained multilingual NMT model
- Compare and evaluate greedy and beam search decoding strategies
- Save translation outputs to csv file
- Generates BLEU scores to evaluate and compare the two decoding strategies


In [ ]:
# Install dependencies
%pip install -r requirements.txt
# !pip install transformers datasets sacrebleu pandas sentencepiece pypinyin epitran hf_xet protobuf ipywidgets

## Load Pretrained mBART-50 NMT Model

In [2]:
import torch
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # for progress bar

In [3]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Import the tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer.src_lang = "yue_Hant"
tokenizer.tgt_lang = "en_XX"

In [5]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

model_name_m2m = "facebook/m2m100_418M"
tokenizer_m2m = M2M100Tokenizer.from_pretrained(model_name_m2m)
model_m2m = M2M100ForConditionalGeneration.from_pretrained(model_name_m2m).to(device)

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

d:\Florence\Homework\UniversityOfWaterloo\FourthYear\CS486\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FY\.cache\huggingface\hub\models--facebook--m2m100_418M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [4]:
# Load MarianMT Model
from transformers import MarianTokenizer, MarianMTModel

model_name_mar = "Helsinki-NLP/opus-mt-yue-en"
tokenizer_mar = MarianTokenizer.from_pretrained(model_name_mar)
model_mar = MarianMTModel.from_pretrained(model_name_mar).to(device)

OSError: Helsinki-NLP/opus-mt-yue-en is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

## Load Dataset from Tatoeba

In [6]:
from datasets import load_dataset
import pandas as pd

In [7]:
import re

# Manually load downloaded .en and .yue files, apply pre-processing to clean the text
def load_parallel_corpus(cantonese_file, english_file, max_lines=None):
    # Open the cantonese and english files
    with open(cantonese_file, encoding='utf-8') as f_yue, open(english_file, encoding='utf-8') as f_en:
        yue_lines = f_yue.readlines()
        en_lines = f_en.readlines()

    # Ensure that there is the same line count
    if max_lines:
        yue_lines = yue_lines[:max_lines]
        en_lines = en_lines[:max_lines]

    assert len(yue_lines) == len(en_lines), "Line count mismatch!"

    #Apply preprocessing to clean the text 
    def clean_text(text):
        # Remove brackets
        text = re.sub(r'\[[^\]]*\]', '', text)
        text = re.sub(r'\([^\)]*\)', '', text)
        
        # Normalize whitespace
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    # Clean each line of text for both cantonese and english files
    yue_lines = [clean_text(line) for line in yue_lines]
    en_lines = [clean_text(line) for line in en_lines]

    # Build the DataFrame and drop empty lines
    df = pd.DataFrame({'cantonese': yue_lines, 'english': en_lines})
    df = df[(df['cantonese'] != '') & (df['english'] != '')].reset_index(drop=True)
    return df

In [8]:
# # Load Tatoeba dataset
df = load_parallel_corpus('en-yue.txt/Tatoeba.en-yue.yue', 'en-yue.txt/Tatoeba.en-yue.en', max_lines=1000)
print(df.head())

             cantonese                                        english
0              我要去瞓覺喇。                         I have to go to sleep.
1  我話唔定做一陣就會放棄，走去瞓晏覺算。       I may give up soon and just nap instead.
2       我不嬲都鍾意啲神秘啲嘅人物。     I always liked mysterious characters more.
3   雖然佢講咗對唔住，但係我都仲係好嬲。  Even though he apologized, I'm still furious.
4               我唯有係等。                               I can only wait.


In [9]:
# Load 1000 lines from OpenSubtitles dataset (downloaded from Opus)
df_Open_Subtitles = load_parallel_corpus('open-subtitles_en-yue.txt/OpenSubtitles.en-yue.yue', 'open-subtitles_en-yue.txt/OpenSubtitles.en-yue.en', max_lines=1000)
print(df_Open_Subtitles.head())

        cantonese                                            english
0              你看                                              Look.
1            笑死我了                                 - That kills me. -
2  來整蠱他吧,讓他見識最佳拍檔  Come on, we'll fix him. Let's give him the old...
3        你還不會用腦嗎?                            Won't you ever grow up?
4              喂喂                                 - What? - Yoo-hoo!


## Define Translation Function

In [10]:
# Translation function with tydm to add a loading progress bar for insights using mBART50 model
def translate(texts, beam=1, batch_size=16):
    translations = []
    # Evaluate the model
    model.eval()
    
    # Devide into batches so that the progress/ percentage is shown too
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                max_length=128,
                num_beams=beam,
                no_repeat_ngram_size=2
            )
        batch_translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        translations.extend(batch_translations)
    return translations

# Translation function with model and tokenizer being passed in
def translate_select_model(texts, tokenizer, model, beam=1, batch_size=16, device="cpu"):
    translations = []
    model.to(device)
    model.eval()
    
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, max_length=128, return_tensors="pt").to(device)
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                max_length=128,
                num_beams=beam,
                no_repeat_ngram_size=2
            )
        batch_translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        translations.extend(batch_translations)
    return translations



## Translate & Compare Decoding Methods

In [11]:
# Run translations for greedy and beam search algorithm using Tatoeba dataset
# df['greedy'] = translate(df['cantonese'].tolist(), beam=1, batch_size=16)
# df['beam'] = translate(df['cantonese'].tolist(), beam=5, batch_size=16)
# df.head()

In [14]:
# Run using M2M
df_m2m = load_parallel_corpus('en-yue.txt/Tatoeba.en-yue.yue', 'en-yue.txt/Tatoeba.en-yue.en', max_lines=1000)
print(df_m2m.head())

df_m2m['greedy'] = translate_select_model(df_m2m['cantonese'].tolist(), tokenizer_m2m, model_m2m, beam=1, batch_size=16, device=device)
df_m2m['beam'] = translate_select_model(df_m2m['cantonese'].tolist(), tokenizer_m2m, model_m2m, beam=5, batch_size=16, device=device)
df_m2m.head()

             cantonese                                        english
0              我要去瞓覺喇。                         I have to go to sleep.
1  我話唔定做一陣就會放棄，走去瞓晏覺算。       I may give up soon and just nap instead.
2       我不嬲都鍾意啲神秘啲嘅人物。     I always liked mysterious characters more.
3   雖然佢講咗對唔住，但係我都仲係好嬲。  Even though he apologized, I'm still furious.
4               我唯有係等。                               I can only wait.


Translating: 100%|██████████| 63/63 [38:20<00:00, 36.51s/it] 


,cantonese,english,greedy,beam
0,我要去瞓覺喇。,I have to go to sleep.,Менің қазір мен қасымда.,Өзiмiкiнiгiпiң кiлiшiрi.
1,我話唔定做一陣就會放棄，走去瞓晏覺算。,I may give up soon and just nap instead.,"Менің қазір, қаңтар мен қазыққа қазанды, менің...","Қандай-ақ, қанақтар мен ұзақ уақытқа дейін қар..."
2,我不嬲都鍾意啲神秘啲嘅人物。,I always liked mysterious characters more.,Менің табылған тазалық тазыққа не айтылады.,Je n’ai pas l’intention de connaître les perso...
3,雖然佢講咗對唔住，但係我都仲係好嬲。,"Even though he apologized, I'm still furious.",Менің қазір мен қаңтар менің жарыққа қадам.,"Қандай-ақ, қазір мен қағидалар мен жарықтармен..."
4,我唯有係等。,I can only wait.,Менің қазір мен қасым.,Je n’ai qu’à attendre.


In [17]:
from sacrebleu import corpus_bleu
from sacrebleu import corpus_chrf

# Function to loop over all  dataframes defined and print out BLEU and CHRF scores
def compute_scores(df, name=""):
    # Prepare hypothesis and reference lists
    greedy_hypotheses = df['greedy'].astype(str).tolist()
    beam_hypotheses = df['beam'].astype(str).tolist()
    references = [df['english'].astype(str).tolist()]

    # Compute BLEU scores
    greedy_bleu = corpus_bleu(greedy_hypotheses, references).score
    beam_bleu = corpus_bleu(beam_hypotheses, references).score

    greedy_chrf = corpus_chrf(greedy_hypotheses, references).score
    beam_chrf = corpus_chrf(beam_hypotheses, references).score

    # Print results
    print(f"\n{name} BLEU Scores:")
    print(f"Greedy BLEU: {greedy_bleu:.2f}")
    print(f"Beam BLEU:   {beam_bleu:.2f}")
    
    print(f"\n{name} CHRF Scores:")
    print(f"Greedy CHRF: {greedy_chrf:.2f}")
    print(f"BEAM CHRF: {beam_chrf:.2f}")
    
    return greedy_bleu, beam_bleu, greedy_chrf, beam_chrf


In [18]:
results_m2m = {}
name = "m2m"
print(f"Evaluating name: {name}")
greedy_bleu, beam_bleu, greedy_chrf, beam_chrf = compute_scores(df_m2m, name)
results_m2m[name] = {"greedy_bleu": greedy_bleu, "beam_bleu": beam_bleu, "greedy_chrf": greedy_chrf, "beam_chrf": beam_chrf}
df_m2m.to_csv(f"trans_res_{name}.csv", index=False)

Evaluating name: m2m

m2m BLEU Scores:
Greedy BLEU: 0.07
Beam BLEU:   0.07

m2m CHRF Scores:
Greedy CHRF: 3.96
BEAM CHRF: 5.22


In [ ]:
# Run using M2M on OpenSubtitles dataset
df_Open_Subtitles['greedy'] = translate_select_model(df_Open_Subtitles['cantonese'].tolist(), tokenizer_m2m, model_m2m, beam=1, batch_size=16, device=device)
df_Open_Subtitles['beam'] = translate_select_model(df_Open_Subtitles['cantonese'].tolist(), tokenizer_m2m, model_m2m, beam=5, batch_size=16, device=device)

Translating:   0%|          | 0/62 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# Run translations for greedy and beam search algorithm using OpenSubtitles dataset
df_OS_mBART = load_parallel_corpus('open-subtitles_en-yue.txt/OpenSubtitles.en-yue.yue', 'open-subtitles_en-yue.txt/OpenSubtitles.en-yue.en', max_lines=1000)
print(df_OS_mBART.head())
df_OS_mBART['greedy'] = translate(df_OS_mBART['cantonese'].tolist(), beam=1, batch_size=16)
df_OS_mBART['beam'] = translate(df_OS_mBART['cantonese'].tolist(), beam=5, batch_size=16)
df_OS_mBART.head()

In [ ]:
# # Run using MarianMT on Tatoeba dataset
# df_Tatoeba = load_parallel_corpus('en-yue.txt/Tatoeba.en-yue.yue', 'en-yue.txt/Tatoeba.en-yue.en', max_lines=1000)
# print(df_Tatoeba.head())

# df_Tatoeba['greedy'] = translate(df_Tatoeba['cantonese'].tolist(), tokenizer_mar, model_mar, beam=1, batch_size=16, device=device)
# df_Tatoeba['beam'] = translate(df_Tatoeba['cantonese'].tolist(), tokenizer_mar, model_mar, beam=5, batch_size=16, device=device)
# df_Tatoeba.head()

In [ ]:
# Run using MarianMT on OpenSubtitles dataset
df_OS_Mar = load_parallel_corpus('open-subtitles_en-yue.txt/OpenSubtitles.en-yue.yue', 'open-subtitles_en-yue.txt/OpenSubtitles.en-yue.en', max_lines=1000)
print(df_OS_Mar.head())

df_OS_Mar['greedy'] = translate(df_OS_Mar['cantonese'].tolist(), tokenizer_mar, model_mar, beam=1, batch_size=16, device=device)
df_OS_Mar['beam'] = translate(df_OS_Mar['cantonese'].tolist(), tokenizer_mar, model_mar, beam=5, batch_size=16, device=device)

        cantonese                                            english
0              你看                                              Look.
1            笑死我了                                 - That kills me. -
2  來整蠱他吧,讓他見識最佳拍檔  Come on, we'll fix him. Let's give him the old...
3        你還不會用腦嗎?                            Won't you ever grow up?
4              喂喂                                 - What? - Yoo-hoo!


NameError: name 'tokenizer_mar' is not defined

## Evaluate algorithms using BLEU scores

In [ ]:
from sacrebleu import corpus_bleu

# Define hypotheses for greedy and beam search (as string lists)
greedy_hypotheses = df['greedy'].astype(str).tolist()
beam_hypotheses = df['beam'].astype(str).tolist()

# Define references as string lists
references = [df['english'].astype(str).tolist()]

# Calculate the bleu score
greedy_bleu = corpus_bleu(greedy_hypotheses, references).score
beam_bleu = corpus_bleu(beam_hypotheses, references).score

# Display bleu score
print(f"Greedy BLEU: {greedy_bleu:.2f}")
print(f"Beam BLEU: {beam_bleu:.2f}")


Greedy BLEU: 7.94
Beam BLEU: 9.21


In [ ]:
# Define a list of (name, dataframe) pairs
dataframes = [
    # ("Tatoeba + mBART", df),
    ("Open Subtitles_mBART", df_OS_mBART),
    ("Tatoeba_m2m", df_m2m),
    ("Open Subtitles_m2m", df_Open_Subtitles),
]

# Compute BLEU and CHRF scores for each DataFrame
results = {}

# Calculate BLEU and CHRF scores for each of the data frames and save to list
for name, df_ in dataframes:
    try:
        print(f"Evaluating name: {name}")
        greedy_bleu, beam_bleu, greedy_chrf, beam_chrf = compute_scores(df_, name)
        results[name] = {"greedy_bleu": greedy_bleu, "beam_bleu": beam_bleu, "greedy_chrf": greedy_chrf, "beam_chrf": beam_chrf}
        df_.to_csv(f"trans_res_{name}.csv", index=False)
    except Exception as e:
        print(f"Error evaluating {name}: {e}")


## Save Translation Comaprison Results to CSV

In [ ]:
# df.to_csv("trans_res_Tatoeba_mBart.csv", index=False)
# df.head()

,cantonese,english,greedy,beam
0,我要去瞓覺喇。,I have to go to sleep.,I'm going to go to the temple.,I'm going to go to the monastery.
1,我話唔定做一陣就會放棄，走去瞓晏覺算。,I may give up soon and just nap instead.,I'm going to say I will give up a fight and go...,I said I was going to do a series I would give...
2,我不嬲都鍾意啲神秘啲嘅人物。,I always liked mysterious characters more.,I'm not a big fan of mysterious characters.,I don't think I've ever heard of a mysterious ...
3,雖然佢講咗對唔住，但係我都仲係好嬲。,"Even though he apologized, I'm still furious.","I'm not sure if I can do it, but I have a good...","I'm not sure if we're going to live together, ..."
4,我唯有係等。,I can only wait.,I'm only a single one.,I'm the only one who can wait.


In [ ]:
# df_Open_Subtitles.to_csv("trans_res_OpenSubtitles_mBart.csv", index=False)
# df_Open_Subtitles.head()

In [ ]:
df_Tatoeba.to_csv("trans_res_Tatoeba_Marian.csv", index=False)
df_Tatoeba.head()

In [ ]:
df_OS_Mar.to_csv("trans_res_OpenSubtitles_Marian.csv", index=False)
df_OS_Mar.head()